In [1]:
import random
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

In [5]:
#!pip install datasets

In [7]:
#!pip install xgboost --upgrade

In [8]:
# df = pd.read_csv("imdb_dataset.csv")
from datasets import load_dataset
dataset = load_dataset("imdb")
df = pd.concat([pd.DataFrame(dataset["train"]),pd.DataFrame(dataset["test"])],axis=0)
print (df.shape)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

(50000, 2)


In [9]:
df.head()

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0


In [10]:
print (df.isnull().sum())

text     0
label    0
dtype: int64


In [11]:
reviews = list(df["text"])
sentiments = list(df["label"])
print (set(sentiments))

{0, 1}


In [12]:
balanced_reviews = reviews
balanced_sentiments = sentiments
print (len(balanced_reviews))
print (len(balanced_sentiments))
print (sum(balanced_sentiments))

50000
50000
25000


In [13]:
# positive_reviews = [r for r, s in zip(reviews, sentiments) if s == "positive"]
# negative_reviews = [r for r, s in zip(reviews, sentiments) if s == "negative"]

# pos_sample_size = int(0.60 * len(positive_reviews))
# neg_sample_size = int(0.60 * len(negative_reviews))

# sampled_positive = random.sample(positive_reviews, pos_sample_size)
# sampled_negative = random.sample(negative_reviews, neg_sample_size)

# balanced_reviews = sampled_positive + sampled_negative
# balanced_sentiments = ["positive"] * pos_sample_size + ["negative"] * neg_sample_size

# combined = list(zip(balanced_reviews, balanced_sentiments))
# random.shuffle(combined)
# balanced_reviews, balanced_sentiments = zip(*combined)

# balanced_reviews = list(balanced_reviews)
# balanced_sentiments = list(balanced_sentiments)

# print (len(balanced_reviews))
# print (len(balanced_sentiments))

In [14]:
X_train, X_temp, y_train, y_temp = train_test_split(balanced_reviews, balanced_sentiments, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
print (len(X_train), len(y_train))
print (len(X_val), len(y_val))
print (len(X_test), len(y_test))

30000 30000
10000 10000
10000 10000


In [15]:
vectorizer = TfidfVectorizer(stop_words="english")
X_train_tfidf = vectorizer.fit_transform(X_train)
print (len(vectorizer.vocabulary_))

82357


In [16]:
vectorizer = TfidfVectorizer(stop_words="english", max_features=10000)
X_train_tfidf = vectorizer.fit_transform(X_train)
print (len(vectorizer.vocabulary_))

10000


In [17]:
vectorizer = TfidfVectorizer(stop_words="english", max_df=0.95, min_df=0.05)
X_train_tfidf = vectorizer.fit_transform(X_train)
print (len(vectorizer.vocabulary_))

258


In [18]:
vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1,2), max_df=0.95, min_df=0.025)
X_train_tfidf = vectorizer.fit_transform(X_train, )
print (len(vectorizer.vocabulary_))

631


In [20]:
#vectorizer.vocabulary_

In [21]:
X_val_tfidf = vectorizer.transform(X_val)
X_test_tfidf = vectorizer.transform(X_test)

In [22]:
X_val_tfidf

<10000x631 sparse matrix of type '<class 'numpy.float64'>'
	with 402473 stored elements in Compressed Sparse Row format>

In [23]:
#X_val_tfidf[9999].data

In [24]:
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

LogisticRegression()

In [25]:
y_train_pred = model.predict(X_train_tfidf)
val_accuracy = accuracy_score(y_train, y_train_pred)
print("Train Accuracy:", val_accuracy)

Train Accuracy: 0.8567666666666667


In [26]:
y_val_pred = model.predict(X_val_tfidf)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)

Validation Accuracy: 0.8487


In [27]:
y_test_pred = model.predict(X_test_tfidf)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

Test Accuracy: 0.8433


In [28]:
model = RandomForestClassifier(n_estimators=30)
model.fit(X_train_tfidf, y_train)

RandomForestClassifier(n_estimators=30)

In [29]:
y_train_pred = model.predict(X_train_tfidf)
val_accuracy = accuracy_score(y_train, y_train_pred)
print("Train Accuracy:", val_accuracy)
y_val_pred = model.predict(X_val_tfidf)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)
y_test_pred = model.predict(X_test_tfidf)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

Train Accuracy: 0.9996666666666667
Validation Accuracy: 0.806
Test Accuracy: 0.8046


In [30]:
model = RandomForestClassifier(n_estimators=50, max_depth=20)
model.fit(X_train_tfidf, y_train)

RandomForestClassifier(max_depth=20, n_estimators=50)

In [31]:
y_train_pred = model.predict(X_train_tfidf)
val_accuracy = accuracy_score(y_train, y_train_pred)
print("Train Accuracy:", val_accuracy)
y_val_pred = model.predict(X_val_tfidf)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)
y_test_pred = model.predict(X_test_tfidf)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

Train Accuracy: 0.9014666666666666
Validation Accuracy: 0.7973
Test Accuracy: 0.7982


In [32]:
model = KNeighborsClassifier(n_neighbors=7)
model.fit(X_train_tfidf, y_train)

KNeighborsClassifier(n_neighbors=7)

In [33]:
y_train_pred = model.predict(X_train_tfidf)
val_accuracy = accuracy_score(y_train, y_train_pred)
print("Train Accuracy:", val_accuracy)
y_val_pred = model.predict(X_val_tfidf)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)
y_test_pred = model.predict(X_test_tfidf)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

Train Accuracy: 0.7962
Validation Accuracy: 0.702
Test Accuracy: 0.7004


In [34]:
model = XGBClassifier(n_estimators=50, max_depth=5)
model.fit(X_train_tfidf, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=50, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [35]:
y_train_pred = model.predict(X_train_tfidf)
val_accuracy = accuracy_score(y_train, y_train_pred)
print("Train Accuracy:", val_accuracy)
y_val_pred = model.predict(X_val_tfidf)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)
y_test_pred = model.predict(X_test_tfidf)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

Train Accuracy: 0.8686666666666667
Validation Accuracy: 0.8126
Test Accuracy: 0.8135


In [36]:
len(y_test_pred)

10000

In [37]:
def ensamble():
    model = LogisticRegression()
    model.fit(X_train_tfidf, y_train)
    y_pred1 = model.predict(X_test_tfidf)

    model = RandomForestClassifier(n_estimators=100, max_depth=20)
    model.fit(X_train_tfidf, y_train)
    y_pred2 = model.predict(X_test_tfidf)

    model = RandomForestClassifier(n_estimators=100, max_depth=20)
    model.fit(X_train_tfidf, y_train)
    y_pred3 = model.predict(X_test_tfidf)

    y_pred = []
    for i in range(len(y_pred1)):
        if y_pred1[i] + y_pred2[i] + y_pred3[i] >= 2:
            y_pred.append(1)
        else:
            y_pred.append(0)

    acc = accuracy_score(y_test, y_pred)
    return acc

In [38]:
acc = ensamble()
print ("Enemble acc:", acc)

Enemble acc: 0.8148
